In [2]:
pip install pyet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import pandas as pd
import glob
from pyet.meteo_utils import extraterrestrial_r
from pyet.combination import pm_fao56

In [12]:
# Función para obtener el valor numérico de un string
def get_number(string):
  number = ''
  for i in string:
    for j in range(48,58):
      if chr(j) == i:
        number += i
  return number

# Constantes para obtener Ra
DEG_TO_RAD_CONSTANT = 0.01745329251  
LAT = -33.3971255
lat_rad = float(LAT) *  DEG_TO_RAD_CONSTANT
elevation = 482.0
# Reduciendo reportes a 7 días
horizon_const = 7

# Cantidad de días en que se tiene el "et0 real" respecto a los reportes de la previsión climática a 7 días, a partir del 01-06-2022
n_muestras = 23

# Ruta
RUTA = '/content/drive/MyDrive/Universidad/Evapotranspiration-Prediction-2022/data_processing/'

In [13]:
# Trabajando con los datasets de previsión climática
csv_files = glob.glob(RUTA+'forecast_data/*.csv')
csv_files.sort()

# 14 days horizon
date = []
t_min = []
t_max = []
rh_day = []
rh_night = []
wind_speed = []
sunshine_duration = []
weather_condition = []
cod_wc = []

horizon_days = []


for i in range(horizon_const):
 date.append([])
 t_min.append([])
 t_max.append([])
 rh_day.append([])
 rh_night.append([])
 wind_speed.append([])
 sunshine_duration.append([])
 weather_condition.append([])
 cod_wc.append([])

# Obtención de valores númericos 
for i in range (len(csv_files)):
  ds = pd.read_csv(csv_files[i])
  for h in range (horizon_const):
    t_min[h].append(float(get_number(ds.t_min[h])))
    t_max[h].append(float(get_number(ds.t_max[h])))
    rh_day[h].append(float(get_number(ds.humidity_day[h])))
    rh_night[h].append(float(get_number(ds.humidity_night[h])))
    wind_speed[h].append(float(get_number(ds.wind_speed[h])))
    weather_condition[h].append(ds.weather_condition[h])
    date[h].append(pd.to_datetime(ds.date[h],format="%d-%m-%Y"))

# Asignando valores numéricos a la condición climática según una aproximación de GUO(2011)
for i in range (len(weather_condition)):
  for j in range(len(weather_condition[i])):
    cond_lluvia = 'lluvia' in weather_condition[i][j].lower()
    cond_chubascos = 'chubascos' in weather_condition[i][j].lower()
    cond_nublado = 'nublado' in weather_condition[i][j].lower()
    cond_soleado = 'soleado' in weather_condition[i][j].lower()
    if (cond_lluvia):
      if (('tarde' or 'mañana') in weather_condition[i][j].lower()):
        cod_wc[i].append(0.1)
      else:
        cod_wc[i].append(0.05)
    elif (cond_chubascos):
      cod_wc[i].append(0.2)
  
    elif (cond_nublado):
      if ('parcialmente' in weather_condition[i][j].lower()):
        cod_wc[i].append(0.7)
      elif ('muy' in weather_condition[i][j].lower()):
        cod_wc[i].append(0.3)
      else:
        cod_wc[i].append(0.5)

    elif (cond_soleado):
      if ('mayormente' in weather_condition[i][j].lower()):
        cod_wc[i].append(0.7)
      else:
        cod_wc[i].append(0.9)

    else:
      cod_wc[i].append(0.5)


# Creación de DataFrames para los 14 días de pronóstico # Indexando Fecha y añadiendo Ra complementaria y ET0 calculado por PMFAO56 con datos de previsión climática
for i in range (horizon_const):
  horizon_days.append(pd.DataFrame({'date':date[i],'tmin':t_min[i],'tmax':t_max[i],'rhday':rh_day[i],'rhnight':rh_night[i],'wind':wind_speed[i],'wc':weather_condition[i],'cod_wc':cod_wc[i]}))
  horizon_days[i] = horizon_days[i].set_index('date')
  horizon_days[i]['ra'] = extraterrestrial_r(horizon_days[i].index,lat_rad)
  horizon_days[i]['et0_pmfao56_raxwc'] = pm_fao56(tmean=(horizon_days[i].tmin+horizon_days[i].tmax)/2,wind = horizon_days[i].wind,rs = (horizon_days[i].ra*horizon_days[i].cod_wc),rh=horizon_days[i].rhday,tmax = horizon_days[i].tmax,tmin = horizon_days[i].tmin,lat = lat_rad,elevation=elevation)
  horizon_days[i] = horizon_days[i].iloc[0:n_muestras+1]

In [9]:
conds = []
for i in range (len(weather_condition)):
  for j in range(len(weather_condition[i])):
    conds.append(weather_condition[i][j])
df_cond = pd.DataFrame(data=conds)
df_cond[0].unique()

array(['Parcialmente nublado', 'Lluvia', 'Muy nublado',
       'Mayormente soleado', 'Nublado', 'Chubascos', 'Lluvia tarde',
       'Soleado', 'Chubascos por la tarde', 'Chubascos matinales'],
      dtype=object)

In [14]:
for i in range(len(horizon_days)):
  horizon_days[i].to_csv(RUTA+'processed_forecast_data/'+str(i+1)+'-Day(s)-forecast-Horizon.csv')